# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras.layers.core import Dropout
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels].values

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels].values

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [ ]:
input_dim = 122
intermediate_dim = 10
latent_dim = 32
batch_size = 1409
hidden_layers = 8
classes = 2
drop_prob = 0.1

class Train:
    def build_vae_model():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu', 
                      kernel_regularizer=keras.regularizers.l2(0.0001),
                      activity_regularizer=keras.regularizers.l1(0.0001))(hidden_encoder)
            
            hidden_encoder = Dropout(rate=drop_prob)(hidden_encoder)

        Train.mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)
        Train.logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            m_e, l_e = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = m_e + K.exp(l_e / 2) * epsilon
            return z

        z = Lambda(get_distrib,name='z_dist')([Train.mean_encoder, Train.logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu", 
                      kernel_regularizer=keras.regularizers.l2(0.0001),
                      activity_regularizer=keras.regularizers.l1(0.0001))(hidden_decoder)
            hidden_decoder = Dropout(rate=drop_prob)(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None, name='vae_output')(hidden_decoder)
        
        #Train.z_ = Input(shape=(latent_dim,))
        hidden_y = Dense(latent_dim, activation='relu', name='softmax_hidden')(z)
        Train.y = Dense(classes, activation='softmax', name='softmax_output')(hidden_y)
        
Train.build_vae_model()

In [ ]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

#features_arr = [4, 16, 32]
#hidden_layers_arr = [2, 6, 10]

features_arr = [4, 16, 32]
hidden_layers_arr = [2, 4, 6]

epoch_arr = [50]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = {}

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h
    
    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    
    optimizer = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
    
    vae_model = Model(inputs = Train.x, outputs = Train.y)
    vae_model.compile(optimizer = optimizer, 
                      loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])   
    
    vae_model.fit(x = x_train[:train_size,:], y = y_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  validation_data = (x_test, y_test),
                  verbose = 1)
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = y_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = y_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    y_test_pred = vae_model.predict(x_test, batch_size=batch_size)
    

    y_pred = y_test_pred #np.argmax(y_test_pred[:,-2:], axis = 1)
    
    curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,0], "Normal_prob":y_pred[:,1]})
    predictions.update({"{}_{}_{}".format(e,f,h):curr_pred})
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)

 
 Current Layer Attributes - epochs:50 hidden layers:2 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 3s - loss: 0.9888 - acc: 0.5345 - val_loss: 0.8699 - val_acc: 0.4308
Epoch 2/50
112720/112720 [==============================] - 1s - loss: 0.7346 - acc: 0.5353 - val_loss: 0.8078 - val_acc: 0.4308
Epoch 3/50
112720/112720 [==============================] - 1s - loss: 0.7164 - acc: 0.5353 - val_loss: 0.7868 - val_acc: 0.4308
Epoch 4/50
112720/112720 [==============================] - 1s - loss: 0.7110 - acc: 0.5353 - val_loss: 0.7763 - val_acc: 0.4308
Epoch 5/50
112720/112720 [==============================] - 1s - loss: 0.7085 - acc: 0.5353 - val_loss: 0.7698 - val_acc: 0.4308
Epoch 6/50
112720/112720 [==============================] - 1s - loss: 0.7072 - acc: 0.5353 - val_loss: 0.7657 - val_acc: 0.4308
Epoch 7/50
112720/112720 [==============================] - 1s - loss: 0.7062 - acc: 0.5353 - val_loss

112720/112720 [==============================] - 1s - loss: 0.6952 - acc: 0.5353 - val_loss: 0.7237 - val_acc: 0.4308
Epoch 12/50
112720/112720 [==============================] - 1s - loss: 0.6951 - acc: 0.5353 - val_loss: 0.7234 - val_acc: 0.4308
Epoch 13/50
112720/112720 [==============================] - 1s - loss: 0.6950 - acc: 0.5353 - val_loss: 0.7231 - val_acc: 0.4308
Epoch 14/50
112720/112720 [==============================] - 1s - loss: 0.6950 - acc: 0.5353 - val_loss: 0.7229 - val_acc: 0.4308
Epoch 15/50
112720/112720 [==============================] - 1s - loss: 0.6949 - acc: 0.5353 - val_loss: 0.7227 - val_acc: 0.4308
Epoch 16/50
112720/112720 [==============================] - 1s - loss: 0.6948 - acc: 0.5353 - val_loss: 0.7226 - val_acc: 0.4308
Epoch 17/50
112720/112720 [==============================] - 1s - loss: 0.6948 - acc: 0.5353 - val_loss: 0.7224 - val_acc: 0.4308
Epoch 18/50
112720/112720 [==============================] - 1s - loss: 0.6948 - acc: 0.5353 - val_los

112720/112720 [==============================] - 1s - loss: 0.6921 - acc: 0.5353 - val_loss: 0.7114 - val_acc: 0.4308
Epoch 23/50
112720/112720 [==============================] - 1s - loss: 0.6921 - acc: 0.5353 - val_loss: 0.7113 - val_acc: 0.4308
Epoch 24/50
112720/112720 [==============================] - 1s - loss: 0.6920 - acc: 0.5353 - val_loss: 0.7113 - val_acc: 0.4308
Epoch 25/50
112720/112720 [==============================] - 1s - loss: 0.6920 - acc: 0.5353 - val_loss: 0.7112 - val_acc: 0.4308
Epoch 26/50
112720/112720 [==============================] - 1s - loss: 0.6920 - acc: 0.5353 - val_loss: 0.7113 - val_acc: 0.4308
Epoch 27/50
112720/112720 [==============================] - 1s - loss: 0.6920 - acc: 0.5353 - val_loss: 0.7112 - val_acc: 0.4308
Epoch 28/50
112720/112720 [==============================] - 1s - loss: 0.6920 - acc: 0.5353 - val_loss: 0.7111 - val_acc: 0.4308
Epoch 29/50
112720/112720 [==============================] - 1s - loss: 0.6920 - acc: 0.5353 - val_los

In [ ]:
scores.sort_values("test_score", ascending=False)

In [ ]:
pd.Panel(predictions).to_pickle("dataset/keras_vae_dense_trained_seperately_nsl_kdd_predictions.pkl")
scores.to_pickle("dataset/keras_vae_dense_trained_seperately_nsl_kdd_scores.pkl")

In [ ]:
pd.Panel(predictions)